Now we parse the content of the JSON file with the QASM circuits

In [177]:
using JSON
using ITensors

# Run only once or gives problems later
# include("utils.jl")
# using .Utils


In [ ]:
json_filename = "subcircuits.json"  
basis_gates = ["h", "rx", "ry", "rz", "rxx", "rzz", "ryy", "cx"]

circuits_data, expval_knitting, coefficients = parse_subcircuits(json_filename);
all_mpo_sequences, all_rhos, observables = mpo_sequence_apply(circuits_data,basis_gates);

In [ ]:
cutoff_val = 1e-15
method_mpo = "naive"

expval_tensors = sum(coefficients[i] * (sum((inner(observables["A"][j],all_rhos["A"][i], cuttoff = cutoff_val, method = method_mpo) * inner(observables["B"][j],all_rhos["B"][i]; cuttoff = cutoff_val, method = method_mpo)) for j in 1:length(observables["A"]))) for i in 1:length(coefficients)).re

discrepancy = abs(round(100*(expval_knitting - expval_tensors)/expval_knitting, digits = 4))

println("""
==============================
• Expected Value Knitting: $(round(expval_knitting, digits = 8))
• Expected Value Tensors: $(round(expval_tensors, digits = 8))
• Discrepancy    : $discrepancy%
==============================
""")


• Expected Value Knitting: 11.760798934094947
• Expected Value Tensors: 11.760798934094808
• Discrepancy    : 0.0%



In [ ]:
os = OpSum()
os += "Y", 1
sites = create_sites(1)
Y_gate = MPO(os, sites)

MPO
[1] ((dim=2|id=333|"Qubit,i1")', (dim=2|id=333|"Qubit,i1"))


In [ ]:
inspect_mpo(Y_gate, sites)


Manual Inspection of MPO:
Site 1 Operator:
ITensor ord=2
Dim 1: (dim=2|id=333|"Qubit,i1")'
Dim 2: (dim=2|id=333|"Qubit,i1")
NDTensors.Dense{ComplexF64, Vector{ComplexF64}}
 2×2
 0.0 + 0.0im  0.0 - 1.0im
 0.0 + 1.0im  0.0 + 0.0im
